In [16]:
#!git clone https://github.com/fany02656/Typicality_project.git

In [17]:
import numpy as np
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms
import os
import matplotlib.pyplot as plt

In [18]:
import torch
model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True, force_reload=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to C:\Users\Yi Wen/.cache\torch\hub\v0.9.0.zip


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [19]:
# from PIL import Image
# from torchvision import transforms
# import os

# path='/content/drive/MyDrive/typicality/Typicality_project/All_Cropped'
# images={}
# directory=['Mountain', 'Beach', 'Mug', 'Banana', 
#            'Car', 'Plane', 'Lighthouse', 'Church']

# for dir in directory:
#   images[dir]=[file for file in os.listdir(path+'/'+dir)]

In [20]:
#if windows
from PIL import Image
from torchvision import transforms
import os

# Load data: if Windows
path='../Image/All_Cropped'

images={}

directory=['Mountain', 'Beach', 
           'Mug', 'Banana', 
           'Car', 'Plane', 
           'Lighthouse', 'Church']

for dir in directory:
    images[dir]=[file for file in os.listdir(path+'/'+dir) if file.endswith(('jpeg', 'jpg'))]

In [21]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # this should be the mean and std for alexnet training dataset
])

In [22]:
# read labels 
with open("../imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# read labels to wordnet synsets
ltw = pd.read_json('../imagenet_label_to_wordnet_synset.json').T
selected_labels = pd.read_csv('../selected_labels.csv',index_col=0) # selected labels

In [23]:
# map type to corresponding label's wordnet id
id_mapping = dict()
for i in selected_labels.index:
    id_mapping[selected_labels.loc[i]['type']] = selected_labels.loc[i]['id']
id_mapping

{'Banana': '07753592-n',
 'Beach': '09428293-n',
 'Car': '02814533-n',
 'Church': '03028079-n',
 'Lighthouse': '02814860-n',
 'Mountain': '09193705-n',
 'Mug': '03063599-n',
 'Plane': '02690373-n'}

In [24]:
# map type to corresponding label's readable label
readable_mapping = dict()
for i in selected_labels.index:
    readable_mapping[selected_labels.loc[i]['type']] = selected_labels.loc[i]['label']
readable_mapping

{'Banana': 'banana',
 'Beach': 'seashore, coast, seacoast, sea-coast',
 'Car': 'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon',
 'Church': 'church, church building',
 'Lighthouse': 'beacon, lighthouse, beacon light, pharos',
 'Mountain': 'alp',
 'Mug': 'coffee mug',
 'Plane': 'airliner'}

In [25]:
#type prob cate+filename
# all_img={'type':[], 'dir':[],'id_labels':[],'readable_labels':[]}
all_img={'type':[], 'dir':[],'prob':[], 'id_label':[], 'readable_label':[]}

In [26]:
for cate, filename in images.items():
    selected_id_label = id_mapping[cate] 
    selected_readable_label = readable_mapping[cate] 
    for f in filename:
        dir=os.path.join(path,cate,f)
        img=Image.open(dir)
        #print(dir)
        input_tensor=preprocess(img)
        input_batch = input_tensor.unsqueeze(0)

        if torch.cuda.is_available():
            input_batch= input_batch.to('cuda')
            model.to('cuda')

        with torch.no_grad():
            output = model(input_batch)
        
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        
        # Get all 1000 categories per image
        prob, catid = torch.topk(probabilities, probabilities.size(0))
#         temp1 = {}
        temp2 = {}
        for i in range(prob.size(0)):
#             temp1[categories[catid[i].item()]] = prob[i].item() # readable labels
            temp2[ltw['id'][catid[i].item()]] = prob[i].item() # wordnet synset id
        
        all_img['dir'].append(dir)
#         all_img['readable_labels'].append(temp1)
#         all_img['id_labels'].append(temp2)
        all_img['readable_label'].append(selected_readable_label)
        all_img['id_label'].append(selected_id_label)
        all_img['prob'].append(temp2[selected_id_label])
        
        
        if 'Mountain' in cate:
            all_img['type'].append('Mountain')
        if 'Beach' in cate:
            all_img['type'].append('Beach')
        if 'Mug' in cate:
            all_img['type'].append('Mug')
        if 'Banana' in cate:
            all_img['type'].append('Banana')
        if 'Car' in cate:
            all_img['type'].append('Car')
        if 'Plane' in cate:
            all_img['type'].append('Plane')
        if 'Lighthouse' in cate:
            all_img['type'].append('Lighthouse')
        if 'Church' in cate:
            all_img['type'].append('Church')

#         print('Typicality of ',f, ' = ',probabilities.max()*100)

In [27]:
all_img_df=pd.DataFrame(all_img)

In [28]:
all_img_df['type'].value_counts()

Beach         16
Banana        16
Church        16
Plane         16
Lighthouse    16
Mountain      16
Car           16
Mug           16
Name: type, dtype: int64

In [29]:
all_img_df.head()

,type,dir,prob,id_label,readable_label
0,Mountain,../Image/All_Cropped\Mountain\1.jpg,0.021666,09193705-n,alp
1,Mountain,../Image/All_Cropped\Mountain\10.jpg,0.073099,09193705-n,alp
2,Mountain,../Image/All_Cropped\Mountain\11.jpg,0.605756,09193705-n,alp
3,Mountain,../Image/All_Cropped\Mountain\12.jpg,0.724987,09193705-n,alp
4,Mountain,../Image/All_Cropped\Mountain\13.jpg,0.078892,09193705-n,alp


In [30]:
# all_img_df.to_csv('googlenet_final.csv')